In [ ]:
import Test.QuickCheck( quickCheck, 
                        Arbitrary( arbitrary ),
                        oneof, elements, sized, (==>), Property )
import Control.Monad -- defines liftM, liftM3, used below
import Data.List
import Data.Char

In [ ]:
p :: [Int] -> Int
p xs = 1 + days `mod` 7
    where hours = sum [x | x <- xs, x > 0]
          days = hours `div` 24

prop_p =
    p [] == 1 &&
    p [-30,-20] == 1 &&
    p [12,-30,7,8,-20] == 2 && 
    p [90,15] == 5 &&
    p [90,-100,23,-20,54] == 7 &&
    p [90,-100,23,-20,55] == 1
    
quickCheck prop_p

In [ ]:
q :: [Int] -> Int
q xs = 1 + days `mod` 7
    where hours [] = 0
          hours (x:xs) | x > 0 = x + hours xs
                       | otherwise = hours xs
          days = hours xs `div` 24
    
prop_q =
    q [] == 1 &&
    q [-30,-20] == 1 &&
    q [12,-30,7,8,-20] == 2 && 
    q [90,15] == 5 &&
    q [90,-100,23,-20,54] == 7 &&
    q [90,-100,23,-20,55] == 1
    
prop_pq l = (q l) == (p l)

quickCheck prop_q
quickCheck prop_pq

In [ ]:
r :: [Int] -> Int
r xs = 1 + days `mod` 7
    where hours = sum $ filter (>0) xs
          days = hours `div` 24
          
prop_r =
    r [] == 1 &&
    r [-30,-20] == 1 &&
    r [12,-30,7,8,-20] == 2 && 
    r [90,15] == 5 &&
    r [90,-100,23,-20,54] == 7 &&
    r [90,-100,23,-20,55] == 1

prop_rp l = (r l) == (p l)

quickCheck prop_r
quickCheck prop_rp

In [ ]:
f :: String -> String
f (s:sx) = [a | (a, b) <- zip (s:sx) sx, a == b]
f "" = ""

prop_f = 
    f "Tennessee" == "nse" &&
    f "bookkeeper" == "oke" &&
    f "llama hooves" == "lo" &&
    f "www.dell.com" == "wwl" &&
    f "ooooh" == "ooo" &&
    f "nNnone here" == "" &&
    f "" == ""
    
quickCheck prop_f

In [ ]:
g :: String -> String

g "" = ""
g [c] = ""
g (c:cx) | c == head cx = c:(g cx)
         | otherwise = g cx
        
prop_g =
    g "Tennessee" == "nse" &&
    g "bookkeeper" == "oke" &&
    g "llama hooves" == "lo" &&
    g "www.dell.com" == "wwl" &&
    g "ooooh" == "ooo" &&
    g "nNnone here" == "" &&
    g "" == ""

prop_fg s = (f s) == (g s) 

quickCheck prop_g
quickCheck prop_fg

In [ ]:
data Regexp = Epsilon
            | Lit Char
            | Seq Regexp Regexp
            | Or Regexp Regexp
        deriving (Eq, Ord)

-- turns a Regexp into a string approximating normal regular expression notation

showRegexp :: Regexp -> String
showRegexp Epsilon = "e"
showRegexp (Lit c) = [toUpper c]
showRegexp (Seq r1 r2) = "(" ++ showRegexp r1 ++ showRegexp r2 ++ ")"
showRegexp (Or r1 r2) = "(" ++ showRegexp r1 ++ "|" ++ showRegexp r2 ++ ")"

-- for checking equality of languages

equal :: Ord a => [a] -> [a] -> Bool
equal xs ys = sort xs == sort ys

-- For QuickCheck

instance Show Regexp where
    show  =  showRegexp

instance Arbitrary Regexp where
  arbitrary = sized expr
    where
      expr n | n <= 0 = oneof [elements [Epsilon]]
             | otherwise = oneof [ liftM Lit arbitrary
                                 , liftM2 Seq subform subform
                                 , liftM2 Or subform subform
                                 ]
             where
               subform = expr (n `div` 2)




r1 = Seq (Lit 'A') (Or (Lit 'A') (Lit 'A'))   -- A(A|A)
r2 = Seq (Or (Lit 'A') Epsilon)
         (Or (Lit 'A') (Lit 'B'))             -- (A|e)(A|B)
r3 = Seq (Or (Lit 'A') (Seq Epsilon
                            (Lit 'A')))
         (Or (Lit 'A') (Lit 'B'))             -- (A|(eA))(A|B)
r4 = Seq (Or (Lit 'A')
             (Seq Epsilon (Lit 'A')))
         (Seq (Or (Lit 'A') (Lit 'B'))
              Epsilon)                        -- (A|(eA))((A|B)e)
r5 = Seq (Seq (Or (Lit 'A')
                  (Seq Epsilon (Lit 'A')))
              (Or Epsilon (Lit 'B')))
         (Seq (Or (Lit 'A') (Lit 'B'))
              Epsilon)                        -- ((A|(eA))(e|B))((A|B)e)
r6 = Seq (Lit 'B')
         (Seq (Lit 'A')
              (Or (Lit 'C') (Lit 'D')))       -- B(A(C|D))

r1' = Or (Seq (Lit 'A') (Lit 'A'))
         (Seq (Lit 'A') (Lit 'A'))            -- (AA)|(AA)
r2' = Or (Seq (Or (Lit 'A') Epsilon)
              (Lit 'A'))
         (Seq (Or (Lit 'A') Epsilon)
              (Lit 'B'))                     -- ((A|e)A)|((A|e)B)
r3' = Or (Seq (Or (Lit 'A')
                  (Seq Epsilon (Lit 'A')))
              (Lit 'A'))
         (Seq (Or (Lit 'A')
                  (Seq Epsilon (Lit 'A')))
              (Lit 'B'))                     -- ((A|(eA))A) | ((A|(eA))B)
r4' = r4                                      -- (A|(eA))((A|B)e)
r5' = Seq (Or (Seq (Or (Lit 'A')
                       (Seq Epsilon (Lit 'A')))
                   Epsilon)
              (Seq (Or (Lit 'A')
                       (Seq Epsilon (Lit 'A')))
                   (Lit 'B')))
          (Seq (Or (Lit 'A') (Lit 'B'))
               Epsilon)                       -- (((A|(eA))e)|((A|(eA))B))((A|B)e)
r6' = Or (Seq (Lit 'B')
              (Seq (Lit 'A') (Lit 'C')))
         (Seq (Lit 'B')
              (Seq (Lit 'A') (Lit 'D')))      -- (B(AC))|(B(AD))

In [ ]:
language :: Regexp -> [String]
language (Epsilon)   = [""]
language (Lit c1)    = [[c1]]
language (Seq r1 r2) = nub [l++r | l <- (language r1), r <- (language r2)]   
language (Or r1 r2)  = nub $ (language r1) ++ (language r2)

prop_language =
    equal (language r1) ["AA"] &&
    equal (language r2) ["AA", "AB", "A", "B"] &&
    equal (language r3) ["AA", "AB"] &&
    equal (language r4) ["AA", "AB"] &&
    equal (language r5) ["AA", "AB", "ABA", "ABB"] &&
    equal (language r6) ["BAC", "BAD"]
        
quickCheck prop_language

In [ ]:
flatten :: Regexp -> Regexp
flatten (Seq r (Or s t)) = Or (flatten $ Seq r s) (flatten $ Seq r t)
flatten (Seq r1 r2) | r1==r1' && r2==r2' = Seq r1 r2
                    | otherwise          = flatten $ Seq r1' r2'
                    where r1' = flatten r1
                          r2' = flatten r2
flatten (Or r1 r2) = Or (flatten r1) (flatten r2)
flatten r = r

prop_flatten = 
    flatten r1 == Or (Seq (Lit 'A') (Lit 'A'))
                     (Seq (Lit 'A') (Lit 'A')) &&
    flatten r2 == Or (Seq (Or (Lit 'A') Epsilon) (Lit 'A'))
                     (Seq (Or (Lit 'A') Epsilon) (Lit 'B')) &&
    flatten r3 == Or (Seq (Or (Lit 'A') (Seq Epsilon (Lit 'A'))) (Lit 'A'))
                     (Seq (Or (Lit 'A') (Seq Epsilon (Lit 'A'))) (Lit 'B')) &&
    flatten r4 == r4 &&
    flatten r5 == Seq (Or (Seq (Or (Lit 'A') (Seq Epsilon (Lit 'A'))) Epsilon)
                          (Seq (Or (Lit 'A') (Seq Epsilon (Lit 'A')))(Lit 'B')))
                      (Seq (Or (Lit 'A') (Lit 'B')) Epsilon) &&
    flatten r6 == Or (Seq (Lit 'B') (Seq (Lit 'A') (Lit 'C')))
                     (Seq (Lit 'B') (Seq (Lit 'A') (Lit 'D')))

quickCheck prop_flatten